<a href="https://colab.research.google.com/github/deokwoo-han/abc/blob/master/10_14_5_1_basic_cnn_ipynb%EC%9D%98_%EC%82%AC%EB%B3%B8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%matplotlib inline

In [ ]:
#!cp ./drive/MyDrive/training_data.zip .

In [ ]:
!ls -l

total 610308
-rw-r--r-- 1 root root 255872552 Oct 14 06:25 1-basic-cnn.h5
-rw-r--r-- 1 root root 255876224 Oct 14 06:54 2-basic-dropout-cnn.h5
drwx------ 5 root root      4096 Oct 14 05:26 drive
drwxr-xr-x 1 root root      4096 Oct  8 13:45 sample_data
drwxr-xr-x 2 root root     36864 Oct 14 05:27 test_data
drwxr-xr-x 2 root root     94208 Oct 14 05:27 training_data
-rw------- 1 root root 113023590 Oct 14 06:42 training_data.zip
drwxr-xr-x 2 root root     36864 Oct 14 05:27 validation_data


In [ ]:
#!unzip training_data.zip #트레인 3000, 테스트, 배리데이션 1000개씩

In [ ]:
import glob #라벨을 점으로 읽어들임, 구분
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras.preprocessing.image import ImageDataGenerator, load_img, img_to_array, array_to_img

In [ ]:
IMG_DIM = (150, 150)#굉장히 작은 사이즈의 이미지들이라서 150, 150픽셀로 정의

train_files = glob.glob('training_data/*')
train_imgs = [img_to_array(load_img(img, target_size=IMG_DIM)) for img in train_files]
train_imgs = np.array(train_imgs)
train_labels = [fn.split('/')[1].split('.')[0].strip() for fn in train_files]

validation_files = glob.glob('validation_data/*')
validation_imgs = [img_to_array(load_img(img, target_size=IMG_DIM)) for img in validation_files]
validation_imgs = np.array(validation_imgs)
validation_labels = [fn.split('/')[1].split('.')[0].strip() for fn in validation_files]

print('Train dataset shape:', train_imgs.shape, 
      '\tValidation dataset shape:', validation_imgs.shape)

KeyboardInterrupt: ignored

In [ ]:
train_imgs_scaled = train_imgs.astype('float32')
validation_imgs_scaled  = validation_imgs.astype('float32')
train_imgs_scaled /= 255
validation_imgs_scaled /= 255#스케일 노멀라이즈, 소수점 값으로

In [ ]:
print(train_imgs[-1].shape)
array_to_img(train_imgs[-1])

In [ ]:
print(train_imgs[0].shape)
array_to_img(train_imgs[0])

In [ ]:
print(train_imgs[1500].shape)
array_to_img(train_imgs[1500])

In [ ]:
print(train_imgs[99].shape)
array_to_img(train_imgs[99])

In [ ]:
batch_size = 50
num_classes = 2
epochs = 100 #150
input_shape = (150, 150, 3)#클래스는 두개

# encode text category labels
from sklearn.preprocessing import LabelEncoder

le = LabelEncoder()#0과 1 값으로
le.fit(train_labels)
train_labels_enc = le.transform(train_labels)
validation_labels_enc = le.transform(validation_labels)#10개 추출

print(train_labels[1495:1505], train_labels_enc[1495:1505])#라벨 인코딩 개와 고양이가 0과 1

### 이미지 증강(인위적으로 데이터를 3배 정도로 늘림)

In [ ]:
from keras.preprocessing.image import ImageDataGenerator

In [ ]:
train_data_gen = ImageDataGenerator(
    featurewise_center=True,
    featurewise_std_normalization=True,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    horizontal_flip=False,
    vertical_flip=True, rescale=1./225
    )#버티컬은 꼬리와 입이 좌우로 바뀔 수 있으니... 스케일을 시켰음 증강

In [ ]:
val_datagen = ImageDataGenerator(rescale=1./255)

In [ ]:
train_datagen.fit(train_imgs_scaled)

In [ ]:
val_datagen.fit(validation_imgs_scaled)

In [ ]:
type(train_datagen)

In [ ]:
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.models import Sequential
from tensorflow.keras import optimizers

### Model Case I 

In [ ]:
model = Sequential()

model.add(Conv2D(16, kernel_size=(3, 3), activation='relu', padding="same",
                 input_shape=input_shape))#모델 쉐이프가 150, 150, 3 이 됨
model.add(Dropout(0.3))
model.add(MaxPooling2D(pool_size=(2, 2)))#첫 필터 사이즈는 16
#다음에 들어가는 인풋 사이즈? 75 75
model.add(Conv2D(64, kernel_size=(3, 3), activation='relu', padding="same"))
model.add(Dropout(0.3))
model.add(MaxPooling2D(pool_size=(2, 2)))
#37이 나올것
model.add(Conv2D(128, kernel_size=(3, 3), activation='relu', padding="same"))
model.add(Dropout(0.3))
model.add(MaxPooling2D(pool_size=(2, 2)))

#18? 
model.add(Flatten())
model.add(Dropout(0.3))#코드 4개 추가#배치 노말라이즈는 딱히... 패널티도 별로 효과가 없을...
model.add(Dense(512, activation='relu'))
model.add(Dense(1, activation='sigmoid'))#안의 파라미터의 값을 작은 수로 만들어 넣어야
#아니면 너무 커서 오래 걸림
#시그모이드 외에 소프트맥스도

#오버피팅 해결하는 드랍 아웃

In [ ]:
model.summary()

In [ ]:
model.compile(loss='binary_crossentropy',
              optimizer= 'ADAM', # optimizers.RMSprop(lr=0.0001)
              metrics=['accuracy'])

#model.summary()
#...
#...

In [ ]:
train_generator = train_datagen.flow(train_imgs, train_labels_enc, batch_size=32)
val_generator = val_datagen.flow(validation_imgs, validation_labels_enc, batch_size=16)

In [ ]:
history = model.fit_generator(train_generator,  epochs=epochs,
                              validation_data=val_generator,  verbose=1)
                    #steps_per_epoch=len(train_imgs) / 32, 
#validation 

In [ ]:
#model.summary()#에포크 100개 #모델의 갯수만 확인할 것, 이해, 피쳐 러닝, 500초

In [ ]:
#history = model.fit(x=train_imgs_scaled, y=train_labels_enc,
#                    validation_data=(validation_imgs_scaled, validation_labels_enc),
#                    batch_size=batch_size,
#                    epochs=epochs,
#                    verbose=1)
#                    #드랍 아웃 때문에 1초 늘어남
#                    #다시 하늘로 솟구치며 드랍아웃도 별 의미가 없는

In [ ]:
f, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 4))#거의 답을 외워버림, 근데 validation은...
t = f.suptitle('Basic CNN Performance', fontsize=12)#퍼포먼스 이름도 바꿨어야
f.subplots_adjust(top=0.85, wspace=0.3)

epoch_list = list(range(1,101))#151
ax1.plot(epoch_list, history.history['accuracy'], label='Accuracy')#라벨 값이 다름 오류
ax1.plot(epoch_list, history.history['val_accuracy'], label='Val_Accuracy')
ax1.set_xticks(np.arange(0, 101, 5))#151
ax1.set_ylabel('Accuracy Value')
ax1.set_xlabel('Epoch')
ax1.set_title('Accuracy')
l1 = ax1.legend(loc="best")

ax2.plot(epoch_list, history.history['loss'], label='Train Loss')
ax2.plot(epoch_list, history.history['val_loss'], label='Validation Loss')
ax2.set_xticks(np.arange(0, 100, 5))#151
ax2.set_ylabel('Loss Value')
ax2.set_xlabel('Epoch')
ax2.set_title('Loss')
l2 = ax2.legend(loc="best")#심각한 오버피팅

In [ ]:
#model.save('1-basic-cnn.h5')
#model.save('2-basic-dropout-cnn.h5')
model.save('3-Aug-cnn.h5')

In [ ]:
#카피가 더 빠름
#!cp ./1-basic-cnn.h5 /content/drive/MyDrive/kosa-2021

In [ ]:
#from google.colab import drive
#drive.mount('/content/drive') #맨 처음

https://github.com/JSJeong-me/CNN-Cats-Dogs/blob/main/3-aug_cnn-1014.ipynb